In [14]:
import pandas as pd
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
# Replace 'your_username', 'your_password', 'your_host', 'your_port', and 'your_database' with the appropriate values
db_params = {'user' : 'postgres',
'password' : 'GeoTerm2023@@',
'host' : '172.16.27.100' , # Usually 'localhost' if the database is running on your local machine
'port' : '5432',  # The default port for PostgreSQL is 5432
'database' : 'ARV_district_buildings'}

quoted_password = quote_plus(db_params['password'])
# Construct the SQLAlchemy connection string
db_connection_string = f"postgresql://{db_params['user']}:{quoted_password}@{db_params['host']}:{db_params['port']}/{db_params['database']}"

engine = create_engine(db_connection_string)

In [69]:
refcat_value = '1502002DD7810B'
def uses_prop(refcat_value):

    # SQL query with a parameter
    

    query_refcat = text(
        """
        SELECT refcat, total_area_wall AS wall, floor_area::numeric(10, 2) AS floor, roof_area::numeric(10, 2) AS roof
        FROM visualization.surface_area
        where refcat = :refcat_value
        ;
        """
    )

    # Execute the query


    with engine.connect() as connection:
        result = connection.execute(query_refcat, {'refcat_value': refcat_value})
        for row in result:
            areabyCadastralcode = {
                row[0]: {
                'wall': float(row[1]),
                'floor': float(row[2]),
                'roof': float(row[3])
            }
        }

    distributionRatio= {}
    ref_use = {}
    query_dwellings = text(
        """    SELECT parcela, referencia_catastral, coef_propiedad_divhor,clave_grupo_uso
            FROM
            catastro.tipo15_su
            WHERE
            parcela IN (SELECT gmlid FROM citydb.cityobject)
            AND parcela = :refcat_value
            ;
        """
            )
    with engine.connect() as connection:
        result = connection.execute(query_dwellings, {'refcat_value': refcat_value})
        for row in result:
            parcela, refcat, coef, clave_grupo_uso = row
        if parcela not in distributionRatio:
            distributionRatio[parcela] = {}
        distributionRatio[parcela][refcat] = float(coef)
        if parcela not in ref_use:
            ref_use[parcela] = {}
        ref_use[parcela][refcat] = str(clave_grupo_uso)

       
    return areabyCadastralcode, distributionRatio, ref_use
    

In [74]:
refcat_value = '1502002DD7810B'
def uses_prop(refcat_value):

    # SQL query with a parameter
    

    query_refcat = text(
        """
        SELECT refcat, total_area_wall AS wall, floor_area::numeric(10, 2) AS floor, roof_area::numeric(10, 2) AS roof
        FROM visualization.surface_area
        where refcat = :refcat_value
        ;
        """
    )

    # Execute the query


    with engine.connect() as connection:
        result = connection.execute(query_refcat, {'refcat_value': refcat_value})
        areabyCadastralcode = {}
        for row in result:
            areabyCadastralcode = {
                row[0]: {
                    'wall': float(row[1]),
                    'floor': float(row[2]),
                    'roof': float(row[3])
                }
            }

    distributionRatio= {}
    ref_use = {}
    query_dwellings = text(
        """    SELECT parcela, referencia_catastral, coef_propiedad_divhor,clave_grupo_uso
            FROM
            catastro.tipo15_su
            WHERE
            parcela IN (SELECT gmlid FROM citydb.cityobject)
            AND parcela = :refcat_value
            ;
        """
    )
    with engine.connect() as connection:
        result = connection.execute(query_dwellings, {'refcat_value': refcat_value})
        for row in result:
            parcela, refcat, coef, clave_grupo_uso = row
            if parcela not in distributionRatio:
                distributionRatio[parcela] = {}
            distributionRatio[parcela][refcat] = float(coef)
            if parcela not in ref_use:
                ref_use[parcela] = {}
            ref_use[parcela][refcat] = str(clave_grupo_uso)

       
    return areabyCadastralcode, distributionRatio, ref_use


In [75]:
uno, dos, tres = uses_prop(refcat_value)

In [76]:
dos

{'1502002DD7810B': {'1502002DD7810B0001WY': 6.271,
  '1502002DD7810B0002EU': 6.271,
  '1502002DD7810B0003RI': 6.229,
  '1502002DD7810B0004TO': 6.229,
  '1502002DD7810B0005YP': 6.271,
  '1502002DD7810B0006UA': 6.271,
  '1502002DD7810B0007IS': 6.229,
  '1502002DD7810B0008OD': 6.229,
  '1502002DD7810B0009PF': 6.271,
  '1502002DD7810B0010IS': 6.271,
  '1502002DD7810B0011OD': 6.229,
  '1502002DD7810B0012PF': 6.229,
  '1502002DD7810B0013AG': 6.271,
  '1502002DD7810B0014SH': 6.271,
  '1502002DD7810B0015DJ': 6.229,
  '1502002DD7810B0016FK': 6.229,
  '1502002DD7810B0017GL': 6.225,
  '1502002DD7810B0018HB': 6.25,
  '1502002DD7810B0019JZ': 6.25,
  '1502002DD7810B0020GL': 6.25,
  '1502002DD7810B0021HB': 6.25,
  '1502002DD7810B0022JZ': 6.25,
  '1502002DD7810B0023KX': 6.25,
  '1502002DD7810B0024LM': 6.25,
  '1502002DD7810B0025BQ': 6.25,
  '1502002DD7810B0026ZW': 6.25,
  '1502002DD7810B0027XE': 6.25,
  '1502002DD7810B0028MR': 6.25,
  '1502002DD7810B0029QT': 6.25,
  '1502002DD7810B0030XE': 6.25,
  '15

In [79]:
class DataProcessor:
    def __init__(self, engine):
        self.engine = engine

    def get_data(self, refcat_value):
        areabyCadastralcode = self._get_areaby_cadastralcode(refcat_value)
        distribution_ratio, ref_use = self._get_distribution_ratio_and_ref_use(refcat_value)
        return areabyCadastralcode, distribution_ratio, ref_use

    def _get_areaby_cadastralcode(self, refcat_value):
        query_refcat = text(
            """
            SELECT refcat, total_area_wall AS wall, floor_area::numeric(10, 2) AS floor, roof_area::numeric(10, 2) AS roof
            FROM visualization.surface_area
            where refcat = :refcat_value
            ;
            """
        )

        with self.engine.connect() as connection:
            result = connection.execute(query_refcat, {'refcat_value': refcat_value})
            areaby_cadastralcode = {}
            for row in result:
                areaby_cadastralcode = {
                    row[0]: {
                        'wall': float(row[1]),
                        'floor': float(row[2]),
                        'roof': float(row[3])
                    }
                }
        return areaby_cadastralcode

    def _get_distribution_ratio_and_ref_use(self, refcat_value):
        distribution_ratio = {}
        ref_use = {}
        query_dwellings = text(
            """    
            SELECT parcela, referencia_catastral, coef_propiedad_divhor,clave_grupo_uso
            FROM catastro.tipo15_su
            WHERE parcela IN (SELECT gmlid FROM citydb.cityobject)
            AND parcela = :refcat_value
            ;
            """
        )

        with self.engine.connect() as connection:
            result = connection.execute(query_dwellings, {'refcat_value': refcat_value})
            for row in result:
                parcela, refcat, coef, clave_grupo_uso = row
                if parcela not in distribution_ratio:
                    distribution_ratio[parcela] = {}
                distribution_ratio[parcela][refcat] = float(coef)
                if parcela not in ref_use:
                    ref_use[parcela] = {}
                ref_use[parcela][refcat] = str(clave_grupo_uso)

        return distribution_ratio, ref_use


In [86]:
data_processor = DataProcessor(engine)
reabyCadastralcode, distributionRatio, ref_use = data_processor.get_data(refcat_value)


In [89]:
ref_use


{'1502002DD7810B': {'1502002DD7810B0001WY': 'V',
  '1502002DD7810B0002EU': 'V',
  '1502002DD7810B0003RI': 'V',
  '1502002DD7810B0004TO': 'V',
  '1502002DD7810B0005YP': 'V',
  '1502002DD7810B0006UA': 'V',
  '1502002DD7810B0007IS': 'V',
  '1502002DD7810B0008OD': 'V',
  '1502002DD7810B0009PF': 'V',
  '1502002DD7810B0010IS': 'V',
  '1502002DD7810B0011OD': 'V',
  '1502002DD7810B0012PF': 'V',
  '1502002DD7810B0013AG': 'V',
  '1502002DD7810B0014SH': 'V',
  '1502002DD7810B0015DJ': 'V',
  '1502002DD7810B0016FK': 'V',
  '1502002DD7810B0017GL': 'V',
  '1502002DD7810B0018HB': 'V',
  '1502002DD7810B0019JZ': 'V',
  '1502002DD7810B0020GL': 'V',
  '1502002DD7810B0021HB': 'V',
  '1502002DD7810B0022JZ': 'V',
  '1502002DD7810B0023KX': 'V',
  '1502002DD7810B0024LM': 'V',
  '1502002DD7810B0025BQ': 'V',
  '1502002DD7810B0026ZW': 'V',
  '1502002DD7810B0027XE': 'V',
  '1502002DD7810B0028MR': 'V',
  '1502002DD7810B0029QT': 'V',
  '1502002DD7810B0030XE': 'V',
  '1502002DD7810B0031MR': 'V',
  '1502002DD7810B0032

In [109]:
from SQL_statements import database_connection, getSurfacesTemp

engine = database_connection.engine



In [110]:
data_processor = getSurfacesTemp.DataProcessor(engine)



In [112]:
areabyCadastralcode, distributionRatio, ref_use = data_processor.get_data(refcat_value)

NameError: name 'text' is not defined

In [107]:
from sqlalchemy import text